# Dynamical and geometrical simulations of EBSD patterns from an Al-steel joint

Håkon Wiik Ånes (hakon.w.anes@ntnu.no)

See the relevant package documentation for more details on the packages used here:
* diffsims: https://diffsims.readthedocs.io/en/latest/
* kikuchipy: https://kikuchipy.org/en/stable/
* hyperspy: hyperspy.org/hyperspy-doc/current/
* pyebsdindex: https://pyebsdindex.readthedocs.io/en/latest/
* orix: https://orix.readthedocs.io/en/stable/

Import required packages and print their versions

In [22]:
# Replace "inline" with "qt5" from the pyqt package for interactive plotting
%matplotlib qt5

from datetime import date
import importlib_metadata
import os

from diffsims.crystallography import ReciprocalLatticeVector
import hyperspy.api as hs
import kikuchipy as kp
import matplotlib.pyplot as plt
import numpy as np
from orix import io
from orix.quaternion import Orientation
import skimage.color as skc


# Directories
# Dataset naming (a-c) = (I-III)
dset = "a"
dir_mp = "/home/hakon/kode/emsoft/emdata/crystal_data"
dir_data = os.path.join("/home/hakon/phd/data/tina", dset)
dir_kp = os.path.join(dir_data, "kp")
dir_patterns = os.path.join(dir_kp, "patterns")
dir_merged = os.path.join(dir_kp, "merged")

# Matplotlib
plt.rcParams.update({"figure.figsize": (5, 5), "font.size": 10})
savefig_kwds = dict(bbox_inches="tight", pad_inches=0, dpi=150)
plot_kwds = dict(
    scalebar_properties=dict(
        box_alpha=0, location="lower right", scale_loc="top"
    )
)

# Various parameters
phase_name_alt = {
    "Al": "al",
    "$\\alpha$-Fe": "ferrite",
    "$\\eta$": "fe2al5",
    "$\\theta$": "fe4al13",
    "$\\alpha_c$": "alfesi"
}
energy = 20

print("Run date: ", date.today())
print("\nSoftware versions\n------------------")
for pkg in [
    "diffsims",
    "hyperspy",
    "kikuchipy",
    "matplotlib",
    "numpy",
    "orix",
    "scikit-image"
]:
    if pkg == "numpy":
        ver = np.__version__
    else:
        ver = importlib_metadata.version(pkg)
    print(pkg, ":", ver)

Run date:  2022-05-31

Software versions
------------------
diffsims : 0.5.dev0
hyperspy : 1.7.0
kikuchipy : 0.6.dev0
matplotlib : 3.5.2
numpy : 1.21.6
orix : 0.9.0
scikit-image : 0.19.2


## Read data

Read processed EBSD patterns into memory

In [2]:
s = kp.load(os.path.join(dir_kp, "pattern_sda.h5"), lazy=False)
sig_shape = s.axes_manager.signal_shape[::-1]
print(s.axes_manager)

<Axes manager, axes: (207, 109|240, 240)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    207 |      0 |       0 |     0.1 |     um 
               y |    109 |      0 |       0 |     0.1 |     um 
---------------- | ------ | ------ | ------- | ------- | ------ 
              dx |    240 |      0 |       0 |       1 |     um 
              dy |    240 |      0 |       0 |       1 |     um 


Read multi-phase map from indexing

In [3]:
xmap = io.load(os.path.join(dir_kp, "merged", "xmap_merged.h5"))
xmap

Phase   Orientations         Name  Space group  Point group  Proper point group       Color
    0   9522 (42.2%)           Al        Fm-3m         m-3m                 432    tab:blue
    1   4753 (21.1%)   $\alpha_c$         Im-3          m-3                  23  tab:orange
    2   5569 (24.7%)  $\alpha$-Fe        Im-3m         m-3m                 432   tab:green
    3    1868 (8.3%)       $\eta$         Cmcm          mmm                 222     tab:red
    4     851 (3.8%)     $\theta$         C2/m          2/m                 112  tab:purple
Properties: mean_intensity, merged_scores, n_best_disori_angle, scores
Scan unit: um

## Select patterns

Select five good quality patterns, one from each phase: we save these to file
as well as them plotted with geometrical simulations on top and the corresponding
best matching simulated pattern.

To find good matching patterns, we navigate them in the phase map with the
normalized cross-correlation (NCC) score in the alpha channel

In [6]:
fig = xmap.plot(overlay="scores", return_figure=True)
maps_phase_ncc = fig.axes[0].get_images()[0].get_array()
maps_phase_ncc = kp.draw.get_rgb_navigator(maps_phase_ncc)
navigator = hs.signals.Signal2D(maps_phase_ncc)

In [20]:
s.plot(navigator=navigator)

Store the (x, y) positions of each pattern and plot them on the NCC score map

In [8]:
xy = {
    "al": (92, 25),
    "alpha_alfesi": (125, 50),
    "ferrite": (98, 97),
    "fe2al5": (162, 79),
    "fe4al13": (191, 73),
}
colors = {
    "al": (0, 143, 159),
    "alpha_alfesi": (117, 216, 185),
    "ferrite": (127, 198, 130),
    "fe2al5": (246, 122, 114),
    "fe4al13": (253, 225, 4),
}

xs, ys = np.array(list(xy.values())).T
colors_arr = np.array(list(colors.values())) / 255

fig, ax = plt.subplots()
ax.imshow(xmap.get_map_data("scores"), cmap="gray")
ax.axis("off")
ax.scatter(xs, ys, marker="*", color=colors_arr, s=100, ec="k")
fig.tight_layout()
fig.savefig(os.path.join(dir_patterns, "selected_patterns.png"), **savefig_kwds)

Extract the selected patterns

In [9]:
n_patterns = len(xy)
s2_arr = np.zeros((n_patterns,) + sig_shape, dtype=s.data.dtype)
for i, (x, y) in enumerate(zip(xs, ys)):
    s2_arr[i] = s.data[y, x]
s2 = kp.signals.EBSD(s2_arr)

/home/hakon/miniconda3/envs/kp-dev/lib/python3.9/site-packages/hyperspy/signal.py:2451: UserWarning: Setting the `metadata` attribute is deprecated and will be removed in HyperSpy 2.0. Use the `set_item` and `add_dictionary` methods of the `metadata` attribute instead.
  warnings.warn(


## Define detector-sample geometry

Read projection centers (PC) from file, determined by PC optimization via Hough
indexing with `PyEBSDIndex`

In [10]:
pcs = np.loadtxt(os.path.join(dir_kp, "cal_pcs.txt"), usecols=[1, 2, 3])
pc = pcs.mean(axis=0)

Define detector-sample geometry

In [11]:
detector = kp.detectors.EBSDDetector(
    shape=sig_shape,
    sample_tilt=70,
    pc=pc,
)
detector

EBSDDetector (240, 240), px_size 1 um, binning 1, tilt 0, azimuthal 0, pc (0.426, 0.19, 0.512)

## Plot geometrical and dynamical simulations of all phases

Extract the flattened 1D indices corresponding to the 2D (x, y) coordinates of
the selected patterns

In [12]:
idx = np.ravel_multi_index((ys, xs), xmap.shape)

### Ferrite

Geometrical simulation

In [13]:
i = 2

# Make reflector list
reflectors = ReciprocalLatticeVector(
    hkl=((0, 1, 1), (0, 0, 2), (1, 1, 2), (0, 2, 2)),
    phase=xmap.phases[i],
)
reflectors = reflectors.symmetrise(unique=True)

# Make simulator
simulator = kp.simulations.KikuchiPatternSimulator(reflectors);

# Get Kikuchi lines on detector
sim = simulator.on_detector(detector, xmap.rotations[idx[i]])

# Phase name
phase_name_i = phase_name_alt[xmap.phases[i].name]

# Plot geometrical simulation on pattern with zone axes
fig = sim.plot(coordinates="detector", pattern=s2.data[i], show_pc=False, return_figure=True, zone_axes_labels=True, zone_axes=False)
fig.axes[0].axis("off")
fig.savefig(os.path.join(dir_patterns, f"pattern_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_patterns, f"pattern_{phase_name_i}_x{xs[i]}_y{ys[i]}.svg"), **savefig_kwds)

# Plot geometrical simulation on pattern without zone axes
fig = sim.plot(coordinates="detector", pattern=s2.data[i], show_pc=False, return_figure=True, zone_axes_labels=False, zone_axes=False)
fig.axes[0].axis("off")
fig.savefig(os.path.join(dir_patterns, f"pattern_no_za_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_patterns, f"pattern_no_za_{phase_name_i}_x{xs[i]}_y{ys[i]}.svg"), **savefig_kwds)

1
2
3
4
5
6
7
8
9
10
11


Dynamical simulation

In [14]:
mp = kp.load(
    os.path.join(dir_mp, phase_name_i, f"{phase_name_i}_mc_mp_20kv.h5"),
    projection="lambert",
    energy=energy
)

In [23]:
sim_dyn = mp.get_patterns(
    xmap.rotations[idx[i]], detector=detector, energy=energy, compute=True
)
plt.imsave(
    os.path.join(dir_patterns, f"pattern_sim_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"),
    sim_dyn.data.squeeze(),
    cmap="gray",
)

Creating a dictionary of (1,) simulated patterns:
[########################################] | 100% Completed |  0.1s


/home/hakon/miniconda3/envs/kp-dev/lib/python3.9/site-packages/hyperspy/signal.py:2451: UserWarning: Setting the `metadata` attribute is deprecated and will be removed in HyperSpy 2.0. Use the `set_item` and `add_dictionary` methods of the `metadata` attribute instead.
  warnings.warn(


### Fe2Al5

Geometrical simulation

In [26]:
i = 3

# Make reflector list
reflectors = ReciprocalLatticeVector(
    hkl=((1, 1, 0), (2, 0, 0), (1, 1, 1), (0, 2, 0), (0, 2, 1), (2, 2, 0), (3, 1, 0), (2, 2, 1), (3, 1, 1), (0, 0, 2), (1, 3, 0), (1, 1, 2)),
    phase=xmap.phases[i],
)
reflectors = reflectors.symmetrise(unique=True)

# Make simulator
simulator = kp.simulations.KikuchiPatternSimulator(reflectors);

# Get Kikuchi lines on detector
sim = simulator.on_detector(detector, xmap.rotations[idx[i]])

# Phase name
phase_name_i = phase_name_alt[xmap.phases[i].name]

# Plot geometrical simulation on pattern with zone axes
fig = sim.plot(coordinates="detector", pattern=s2.data[i], show_pc=False, return_figure=True, zone_axes_labels=True, zone_axes=False)
fig.axes[0].axis("off")
fig.savefig(os.path.join(dir_patterns, f"pattern_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_patterns, f"pattern_{phase_name_i}_x{xs[i]}_y{ys[i]}.svg"), **savefig_kwds)

# Plot geometrical simulation on pattern without zone axes
fig = sim.plot(coordinates="detector", pattern=s2.data[i], show_pc=False, return_figure=True, zone_axes_labels=False, zone_axes=False)
fig.axes[0].axis("off")
fig.savefig(os.path.join(dir_patterns, f"pattern_no_za_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_patterns, f"pattern_no_za_{phase_name_i}_x{xs[i]}_y{ys[i]}.svg"), **savefig_kwds)

1
2
3
4
5
6
7
8
9
10
11


Dynamical simulation

In [27]:
mp = kp.load(
    os.path.join(dir_mp, phase_name_i, f"{phase_name_i}_mc_mp_20kv.h5"),
    projection="lambert",
    energy=energy
)

In [28]:
sim_dyn = mp.get_patterns(
    xmap.rotations[idx[i]], detector=detector, energy=energy, compute=True
)
plt.imsave(
    os.path.join(dir_patterns, f"pattern_sim_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"),
    sim_dyn.data.squeeze(),
    cmap="gray",
)

Creating a dictionary of (1,) simulated patterns:
[########################################] | 100% Completed |  0.1s


/home/hakon/miniconda3/envs/kp-dev/lib/python3.9/site-packages/hyperspy/signal.py:2451: UserWarning: Setting the `metadata` attribute is deprecated and will be removed in HyperSpy 2.0. Use the `set_item` and `add_dictionary` methods of the `metadata` attribute instead.
  warnings.warn(


### Fe4Al13

Geometrical simulation

In [29]:
i = 4

# Make reflector list
reflectors = ReciprocalLatticeVector(
    hkl=(
        (6, 2, 0),
        (0, 4, 0),
        (6, 2, -3),
        (0, 2, 5),
        (4, 2, -5),
        (4, 2, 3),
        (0, 0, 3),
        (6, 0, -5),
        (2, 2, 0),
        (2, 0, 5),
        (2, 0, -6),
        (2, 0, -3),
        (6, 0, 2),
    ),
    phase=xmap.phases[i],
)
reflectors = reflectors.symmetrise(unique=True)

# Make simulator
simulator = kp.simulations.KikuchiPatternSimulator(reflectors);

# Get Kikuchi lines on detector
sim = simulator.on_detector(detector, xmap.rotations[idx[i]])

# Phase name
phase_name_i = phase_name_alt[xmap.phases[i].name]

# Plot geometrical simulation on pattern with zone axes
fig = sim.plot(coordinates="detector", pattern=s2.data[i], show_pc=False, return_figure=True, zone_axes_labels=True, zone_axes=False)
fig.axes[0].axis("off")
fig.savefig(os.path.join(dir_patterns, f"pattern_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_patterns, f"pattern_{phase_name_i}_x{xs[i]}_y{ys[i]}.svg"), **savefig_kwds)

# Plot geometrical simulation on pattern without zone axes
fig = sim.plot(coordinates="detector", pattern=s2.data[i], show_pc=False, return_figure=True, zone_axes_labels=False, zone_axes=False)
fig.axes[0].axis("off")
fig.savefig(os.path.join(dir_patterns, f"pattern_no_za_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_patterns, f"pattern_no_za_{phase_name_i}_x{xs[i]}_y{ys[i]}.svg"), **savefig_kwds)

1
2
3
4
5
6
7
8
9
10
11


Dynamical simulation

In [30]:
mp = kp.load(
    os.path.join(dir_mp, phase_name_i, f"{phase_name_i}_mc_mp_20kv.h5"),
    projection="lambert",
    energy=energy
)

In [31]:
sim_dyn = mp.get_patterns(
    xmap.rotations[idx[i]], detector=detector, energy=energy, compute=True
)
plt.imsave(
    os.path.join(dir_patterns, f"pattern_sim_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"),
    sim_dyn.data.squeeze(),
    cmap="gray",
)

Creating a dictionary of (1,) simulated patterns:
[########################################] | 100% Completed |  0.1s


/home/hakon/miniconda3/envs/kp-dev/lib/python3.9/site-packages/hyperspy/signal.py:2451: UserWarning: Setting the `metadata` attribute is deprecated and will be removed in HyperSpy 2.0. Use the `set_item` and `add_dictionary` methods of the `metadata` attribute instead.
  warnings.warn(


### Alpha-AlFeSi

Geometrical simulation

In [32]:
i = 1

# Make reflector list
reflectors = ReciprocalLatticeVector(
    hkl=(
        (2, 3, 5),
        (3, 0, 5),
        (3, 5, 8),
        (0, 0, 6)
    ),
    phase=xmap.phases[i],
)
reflectors = reflectors.symmetrise(unique=True)

# Make simulator
simulator = kp.simulations.KikuchiPatternSimulator(reflectors);

# Get Kikuchi lines on detector
sim = simulator.on_detector(detector, xmap.rotations[idx[i]])

# Phase name
phase_name_i = phase_name_alt[xmap.phases[i].name]

# Plot geometrical simulation on pattern with zone axes
fig = sim.plot(coordinates="detector", pattern=s2.data[i], show_pc=False, return_figure=True, zone_axes_labels=True, zone_axes=False)
fig.axes[0].axis("off")
fig.savefig(os.path.join(dir_patterns, f"pattern_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_patterns, f"pattern_{phase_name_i}_x{xs[i]}_y{ys[i]}.svg"), **savefig_kwds)

# Plot geometrical simulation on pattern without zone axes
fig = sim.plot(coordinates="detector", pattern=s2.data[i], show_pc=False, return_figure=True, zone_axes_labels=False, zone_axes=False)
fig.axes[0].axis("off")
fig.savefig(os.path.join(dir_patterns, f"pattern_no_za_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_patterns, f"pattern_no_za_{phase_name_i}_x{xs[i]}_y{ys[i]}.svg"), **savefig_kwds)

1
2
3
4
5
6
7
8
9
10
11


Dynamical simulation

In [34]:
mp = kp.load(
    os.path.join(dir_mp, "alpha_alfesi2", "alpha_alfesi2_mc_mp_20kv.h5"),
    projection="lambert",
    energy=energy
)

In [35]:
sim_dyn = mp.get_patterns(
    xmap.rotations[idx[i]], detector=detector, energy=energy, compute=True
)
plt.imsave(
    os.path.join(dir_patterns, f"pattern_sim_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"),
    sim_dyn.data.squeeze(),
    cmap="gray",
)

Creating a dictionary of (1,) simulated patterns:
[########################################] | 100% Completed |  0.1s


/home/hakon/miniconda3/envs/kp-dev/lib/python3.9/site-packages/hyperspy/signal.py:2451: UserWarning: Setting the `metadata` attribute is deprecated and will be removed in HyperSpy 2.0. Use the `set_item` and `add_dictionary` methods of the `metadata` attribute instead.
  warnings.warn(


### Al

Geometrical simulation

In [36]:
i = 0

# Make reflector list
reflectors = ReciprocalLatticeVector(
    hkl=((1, 1, 1), (2, 0, 0), (2, 2, 0), (3, 1, 1)),
    phase=xmap.phases[i],
)
reflectors = reflectors.symmetrise(unique=True)

# Make simulator
simulator = kp.simulations.KikuchiPatternSimulator(reflectors);

# Get Kikuchi lines on detector
sim = simulator.on_detector(detector, xmap.rotations[idx[i]])

# Phase name
phase_name_i = phase_name_alt[xmap.phases[i].name]

# Plot geometrical simulation on pattern with zone axes
fig = sim.plot(coordinates="detector", pattern=s2.data[i], show_pc=False, return_figure=True, zone_axes_labels=True, zone_axes=False)
fig.axes[0].axis("off")
fig.savefig(os.path.join(dir_patterns, f"pattern_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_patterns, f"pattern_{phase_name_i}_x{xs[i]}_y{ys[i]}.svg"), **savefig_kwds)

# Plot geometrical simulation on pattern without zone axes
fig = sim.plot(coordinates="detector", pattern=s2.data[i], show_pc=False, return_figure=True, zone_axes_labels=False, zone_axes=False)
fig.axes[0].axis("off")
fig.savefig(os.path.join(dir_patterns, f"pattern_no_za_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"), **savefig_kwds)
fig.savefig(os.path.join(dir_patterns, f"pattern_no_za_{phase_name_i}_x{xs[i]}_y{ys[i]}.svg"), **savefig_kwds)

1
2
3
4
5
6
7
8
9
10
11


Dynamical simulation

In [37]:
mp = kp.load(
    os.path.join(dir_mp, phase_name_i, f"{phase_name_i}_mc_mp_20kv.h5"),
    projection="lambert",
    energy=energy
)

In [38]:
sim_dyn = mp.get_patterns(
    xmap.rotations[idx[i]], detector=detector, energy=energy, compute=True
)
plt.imsave(
    os.path.join(dir_patterns, f"pattern_sim_{phase_name_i}_x{xs[i]}_y{ys[i]}.png"),
    sim_dyn.data.squeeze(),
    cmap="gray",
)

Creating a dictionary of (1,) simulated patterns:
[########################################] | 100% Completed |  0.1s


/home/hakon/miniconda3/envs/kp-dev/lib/python3.9/site-packages/hyperspy/signal.py:2451: UserWarning: Setting the `metadata` attribute is deprecated and will be removed in HyperSpy 2.0. Use the `set_item` and `add_dictionary` methods of the `metadata` attribute instead.
  warnings.warn(
